In [ ]:
# =============================
# ✅ STEP 0: Install Dependencies
# =============================
!pip install -q transformers datasets peft accelerate evaluate gradio bert-score

# =============================
# ✅ STEP 1: Imports & Setup
# =============================
import os, json, random, numpy as np, re
from typing import List, Dict, Any

import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
    DataCollatorForSeq2Seq,
    set_seed,
)
from peft import LoraConfig, get_peft_model
import evaluate
import gradio as gr

# Check GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Using device: {device}")

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
set_seed(SEED)

# =============================
# ✅ STEP 2: Build Dataset
# =============================
bug_templates = [
    ("int main(){int arr[3]; arr[3] = 7; return 0;}",
     "Bug: Out-of-bounds array access at arr[3]. Fix: Valid indices are 0–2."),
    ("char *s=NULL; *s='a';",
     "Bug: Dereferencing NULL pointer. Fix: Allocate memory before dereference."),
    ("int *p=malloc(sizeof(int)); if(p) return 0;",
     "Bug: Memory leak. Fix: Call free(p) on all successful allocation paths."),
    ("int x; if(x==1){printf(\"ok\");}",
     "Bug: Use of uninitialized variable x. Fix: Initialize x before use."),
    ("char buf[4]; strcpy(buf,\"test\");",
     "Bug: Buffer overflow (5 bytes into 4). Fix: Use strncpy with size or increase buffer."),
    ("int *p=malloc(sizeof(int)); free(p); free(p);",
     "Bug: Double free. Fix: Avoid freeing the same pointer twice; set to NULL after free."),
    ("char buf[8]; gets(buf);",
     "Bug: gets() is unsafe and can overflow. Fix: Use fgets with size limit."),
    ("int main(){for(int i=0;i<10;i--){printf(\"%d\",i);} }",
     "Bug: Infinite loop due to wrong update direction. Fix: Use i++ to reach termination."),
    ("FILE *f=fopen(\"file.txt\",\"r\"); fclose(f); fclose(f);",
     "Bug: Double fclose. Fix: Ensure fclose is called once per opened file."),
    ("int a=2147483647; int b=a+1;",
     "Bug: Signed integer overflow when adding 1 to INT_MAX. Fix: Use wider type or check bounds."),
]

clean_templates = [
    ("int main(){int x=0; printf(\"%d\", x); return 0;}",
     "No bug: Code initializes and prints x correctly."),
    ("char s[6]; strcpy(s, \"hi\");",
     "No bug: Buffer is large enough for the copied string and null terminator."),
    ("int arr[3]={1,2,3}; for(int i=0;i<3;i++){ printf(\"%d\", arr[i]); }",
     "No bug: Proper array bounds in loop."),
    ("FILE *f=fopen(\"file.txt\",\"w\"); if(f){ fprintf(f,\"ok\"); fclose(f);} return 0;",
     "No bug: File opened, used, and closed safely."),
]

examples = []
TOTAL_EXAMPLES = 200
half = TOTAL_EXAMPLES // 2

for _ in range(half):
    code, bugfix = random.choice(bug_templates)
    examples.append({"prompt": f"Find bug in this C code:\n\n{code}", "response": bugfix})

for _ in range(half):
    code, expl = random.choice(clean_templates)
    examples.append({"prompt": f"Find bug in this C code:\n\n{code}", "response": expl})

random.shuffle(examples)
train_size = int(0.8 * len(examples))
train_data, test_data = examples[:train_size], examples[train_size:]

print(f"✅ Dataset ready: {len(train_data)} train, {len(test_data)} test")

# =============================
# ✅ STEP 3: Model + LoRA Setup
# =============================
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    target_modules=["q", "k", "v", "o"],
    task_type="SEQ_2_SEQ_LM",
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# =============================
# ✅ STEP 4: Preprocessing
# =============================
INPUT_MAX_LEN = 256
OUTPUT_MAX_LEN = 128

def preprocess_batch(batch):
    model_inputs = tokenizer(batch["prompt"], truncation=True, padding="max_length", max_length=INPUT_MAX_LEN)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(batch["response"], truncation=True, padding="max_length", max_length=OUTPUT_MAX_LEN)
    labels_ids = [[(tid if tid != tokenizer.pad_token_id else -100) for tid in seq] for seq in labels["input_ids"]]
    model_inputs["labels"] = labels_ids
    return model_inputs

train_dataset = Dataset.from_list(train_data).map(preprocess_batch, batched=True, remove_columns=["prompt", "response"])
test_dataset = Dataset.from_list(test_data).map(preprocess_batch, batched=True, remove_columns=["prompt", "response"])

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# =============================
# ✅ STEP 5: Training Config
# =============================
training_args = TrainingArguments(
    output_dir="./gen_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=torch.cuda.is_available(),
    predict_with_generate=True,
    generation_max_length=OUTPUT_MAX_LEN,
    logging_dir="./gen_logs",
    logging_steps=20,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="rougeL",
    greater_is_better=True,
    seed=SEED,
)

# =============================
# ✅ STEP 6: Metrics
# =============================
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

def normalize_text_list(texts):
    return [re.sub(r"\s+", " ", t).strip() for t in texts]

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = normalize_text_list(decoded_preds)
    decoded_labels = normalize_text_list(decoded_labels)
    rouge_res = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    bs = bertscore.compute(predictions=decoded_preds, references=decoded_labels, lang="en")
    metrics = dict(rouge_res)
    metrics["bertscore_f1"] = float(np.mean(bs["f1"]))
    return metrics

# =============================
# ✅ STEP 7: Train
# =============================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)

trainer.train()
eval_results = trainer.evaluate()
print("📊 Evaluation Results:", json.dumps(eval_results, indent=2))

# =============================
# ✅ STEP 8: Inference + Gradio UI
# =============================
def generate_explanation(code_snippet):
    prompt = f"Find bug in this C code:\n\n{code_snippet}"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=INPUT_MAX_LEN).to(device)
    outputs = model.generate(**inputs, max_length=OUTPUT_MAX_LEN, num_beams=4)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Quick test
print("\n🔹 Sample Test:\n", test_data[0]["prompt"])
print("\n💡 Prediction:\n", generate_explanation(test_data[0]["prompt"]))

# Launch Gradio UI
gr.Interface(
    fn=generate_explanation,
    inputs=gr.Textbox(lines=12, placeholder="Paste C code here..."),
    outputs="text",
    title="C Bug Detector + Explainer (T5 + LoRA)",
    description="Detects if there is a bug and explains the fix."
).launch(share=True)
